In [8]:
import pandas as pd
import fitdecode
import os
import warnings

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore')

#### Pasta contendo os dados brutos de cada arquivo

In [9]:
# Caminho da pasta que terá iteração nos seus arquivos
folder_path = '.../running_inactivity_prediction/athletes_activities/athlete5'

#### Extraindo a a data da atividade, distância, tempo da ativdade, pace e velocidade dos dados brutos e passando para um dicionário

In [10]:
df_athlete_dict = {
    'activity_date': [],    
    'activity_type': [],          
    'total_distance (km)': [],        
    'total_time (min)': [],           
    'pace (min/km)': [],              
    'speed (km/h)': []             
}

# Itera por cada arquivo na pasta especificada
for filename in os.listdir(folder_path):
    # Concatena o caminho da pasta com o nome do arquivo para obter o caminho completo
    full_path = os.path.join(folder_path, filename)

    # Abre e lê o arquivo .fit usando fitdecode
    with fitdecode.FitReader(full_path) as fit_file:
        # Itera sobre cada quadro de dados no arquivo .fit
        for frame in fit_file:
            # Verifica se o quadro é do tipo dado (DATA FRAME)
            if frame.frame_type == fitdecode.FIT_FRAME_DATA:
                # Verifica se o quadro corresponde a uma sessão (session)
                if frame.name == 'session':
                    # Itera sobre cada campo de dados na sessão
                    for field in frame.fields:
                        # Verifica se o campo é 'start_time' e armazena a data da atividade
                        if field.name == 'start_time':
                            df_athlete_dict['activity_date'].append(field.value)
                        
                        # Verifica se o campo é 'sport' e armazena o tipo da atividade
                        elif field.name == 'sport':
                            df_athlete_dict['activity_type'].append(field.value)

                        # Verifica se o campo é 'total_distance' e armazena a distância total (convertida para km)
                        elif field.name == 'total_distance':
                            total_dist_km = field.value / 1000
                            df_athlete_dict['total_distance (km)'].append(round(total_dist_km, 2))

                        # Verifica se o campo é 'total_timer_time' e armazena o tempo total (convertido para minutos)
                        elif field.name == 'total_timer_time':
                            total_time_min = field.value // 60
                            df_athlete_dict['total_time (min)'].append(total_time_min)
    
    # Calcula e armazena o pace (min/km) para a atividade atual
    if total_dist_km == 0:  # Se a distância for zero, o pace também é definido como zero
        df_athlete_dict['pace (min/km)'].append(0)
    else:  # Caso contrário, calcula o pace como o tempo total dividido pela distância total
        df_athlete_dict['pace (min/km)'].append(round(total_time_min / total_dist_km, 2))

    # Calcula e armazena a velocidade média (km/h) para a atividade atual
    df_athlete_dict['speed (km/h)'].append(round(total_dist_km / (total_time_min / 60), 2))

#### Criando um Dataframe novo a partir do dicionário

In [11]:
df_athlete_activities = pd.DataFrame(df_athlete_dict)

#### Convertendo a coluna de data para o fuso horário local

In [12]:
df_athlete_activities['activity_date'] = df_athlete_activities['activity_date'].dt.tz_convert('America/Sao_Paulo').dt.tz_localize(None).dt.normalize()

#### Ordenando a data da menor pra maior e resetando index

In [13]:
df_athlete_activities.sort_values('activity_date', ascending=True, inplace=True)
df_athlete_activities.reset_index(drop=True, inplace=True)

In [14]:
df_athlete_activities

,activity_date,activity_type,total_distance (km),total_time (min),pace (min/km),speed (km/h)
0,2021-11-07,running,7.02,39.0,5.56,10.79
1,2021-11-11,running,6.72,45.0,6.70,8.96
2,2021-11-15,running,42.01,192.0,4.57,13.13
3,2021-11-17,swimming,0.30,14.0,46.67,1.29
4,2021-11-18,running,6.75,40.0,5.93,10.12
...,...,...,...,...,...,...
390,2024-06-20,training,0.00,15.0,0.00,0.00
391,2024-06-20,training,0.00,26.0,0.00,0.00
392,2024-06-21,training,0.00,18.0,0.00,0.00
393,2024-06-23,running,14.91,90.0,6.04,9.94
